In [21]:
bucket = "nikki-retail-data"

!gsutil cp gs://{bucket}/Walmart/features.csv features.csv
!gsutil cp gs://{bucket}/Walmart/sales.csv sales.csv
!gsutil cp gs://{bucket}/Walmart/stores.csv stores.csv

Copying gs://nikki-retail-data/Walmart/features.csv...
/ [1 files][  131.0 B/  131.0 B]                                                
Operation completed over 1 objects/131.0 B.                                      
Copying gs://nikki-retail-data/Walmart/sales.csv...
/ [1 files][  100.0 B/  100.0 B]                                                
Operation completed over 1 objects/100.0 B.                                      
Copying gs://nikki-retail-data/Walmart/stores.csv...
/ [1 files][   38.0 B/   38.0 B]                                                
Operation completed over 1 objects/38.0 B.                                       


In [22]:
# Install required libraries
!pip install pyspark google-cloud-bigquery pandas

In [23]:
from google.colab import auth
auth.authenticate_user()

In [24]:
!pip install pyspark google-cloud-bigquery


In [25]:
from google.colab import auth
auth.authenticate_user()


In [26]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailSalesForecast") \
    .getOrCreate()


In [27]:
features = spark.read.csv("features.csv", header=True, inferSchema=True)
sales = spark.read.csv("sales.csv", header=True, inferSchema=True)
stores = spark.read.csv("stores.csv", header=True, inferSchema=True)

In [28]:
features.show()
sales.show()
stores.show()

+-----+----------+-----------+----------+--------+------------+
|Store|      Date|Temperature|Fuel_Price|     CPI|Unemployment|
+-----+----------+-----------+----------+--------+------------+
|    1|2010-02-05|      42.31|     2.572|211.0964|       8.106|
|    1|2010-02-12|      38.51|     2.548|211.2422|       8.106|
+-----+----------+-----------+----------+--------+------------+

+-----+----+----------+------------+---------+
|Store|Dept|      Date|Weekly_Sales|IsHoliday|
+-----+----+----------+------------+---------+
|    1|   1|2010-02-05|   1643691.0|    false|
|    1|   1|2010-02-12|   1641957.0|     true|
+-----+----+----------+------------+---------+

+-----+----+------+
|Store|Type|  Size|
+-----+----+------+
|    1|   A|151315|
|    2|   A|202307|
+-----+----+------+



In [29]:
from pyspark.sql.functions import to_date, col

# Convert date strings to proper Date type
features = features.withColumn("Date", to_date("Date", "yyyy-MM-dd"))
sales = sales.withColumn("Date", to_date("Date", "yyyy-MM-dd"))

# Join all three datasets
df = sales.join(features, ["Store", "Date"]).join(stores, ["Store"])

# View the joined schema and a sample of data
df.printSchema()
df.show(5)

root
 |-- Store: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Dept: integer (nullable = true)
 |-- Weekly_Sales: double (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Temperature: double (nullable = true)
 |-- Fuel_Price: double (nullable = true)
 |-- CPI: double (nullable = true)
 |-- Unemployment: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Size: integer (nullable = true)

+-----+----------+----+------------+---------+-----------+----------+--------+------------+----+------+
|Store|      Date|Dept|Weekly_Sales|IsHoliday|Temperature|Fuel_Price|     CPI|Unemployment|Type|  Size|
+-----+----------+----+------------+---------+-----------+----------+--------+------------+----+------+
|    1|2010-02-05|   1|   1643691.0|    false|      42.31|     2.572|211.0964|       8.106|   A|151315|
|    1|2010-02-12|   1|   1641957.0|     true|      38.51|     2.548|211.2422|       8.106|   A|151315|
+-----+----------+----+------------+------

In [30]:
from google.colab import auth
auth.authenticate_user()


In [31]:
bucket = "nikki-retail-data"

!gsutil cp gs://{bucket}/Walmart/features.csv features.csv
!gsutil cp gs://{bucket}/Walmart/sales.csv sales.csv
!gsutil cp gs://{bucket}/Walmart/stores.csv stores.csv


Copying gs://nikki-retail-data/Walmart/features.csv...
/ [1 files][  131.0 B/  131.0 B]                                                
Operation completed over 1 objects/131.0 B.                                      
Copying gs://nikki-retail-data/Walmart/sales.csv...
/ [1 files][  100.0 B/  100.0 B]                                                
Operation completed over 1 objects/100.0 B.                                      
Copying gs://nikki-retail-data/Walmart/stores.csv...
/ [1 files][   38.0 B/   38.0 B]                                                
Operation completed over 1 objects/38.0 B.                                       


In [32]:
from pyspark.sql.functions import to_date, col

features = spark.read.csv("features.csv", header=True, inferSchema=True)
sales = spark.read.csv("sales.csv", header=True, inferSchema=True)
stores = spark.read.csv("stores.csv", header=True, inferSchema=True)

features = features.withColumn("Date", to_date("Date", "yyyy-MM-dd"))
sales = sales.withColumn("Date", to_date("Date", "yyyy-MM-dd"))

df = sales.join(features, ["Store", "Date"]).join(stores, ["Store"])


In [33]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

input_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

final_data = assembler.transform(df).select("features", col("Weekly_Sales").alias("label"))


In [34]:
from pyspark.sql import SparkSession

# Restart your Spark session
spark = SparkSession.builder \
    .appName("RetailSalesForecasting") \
    .getOrCreate()


In [35]:
features = spark.read.csv("features.csv", header=True, inferSchema=True)
sales = spark.read.csv("sales.csv", header=True, inferSchema=True)
stores = spark.read.csv("stores.csv", header=True, inferSchema=True)


In [36]:
features = features.withColumn("Date", to_date("Date", "yyyy-MM-dd"))
sales = sales.withColumn("Date", to_date("Date", "yyyy-MM-dd"))


In [37]:
df = sales.join(features, ["Store", "Date"]).join(stores, ["Store"])


In [38]:
input_cols = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
assembler = VectorAssembler(inputCols=input_cols, outputCol="features")


In [39]:
final_data = assembler.transform(df).select("features", col("Weekly_Sales").alias("label"))


In [40]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RetailSalesForecasting") \
    .getOrCreate()


In [41]:
from pyspark.ml.regression import LinearRegression

# Initialize the model
lr = LinearRegression(featuresCol="features", labelCol="label")

# Train the model
lr_model = lr.fit(final_data)

# Print model coefficients
print("Coefficients: ", lr_model.coefficients)
print("Intercept: ", lr_model.intercept)


Coefficients:  [152.10526316426416,24083.333336564763,-3964.334707182691,0.0]
Intercept:  2412169.8780554


In [42]:
# Make predictions
predictions = lr_model.transform(final_data)

# Evaluate model
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction")

rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})
r2 = evaluator.evaluate(predictions, {evaluator.metricName: "r2"})

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R²: {r2}")


RMSE: 2.0454175357892628e-07
MAE: 2.0454172044992447e-07
R²: 1.0
